In [1]:
import imageio
import torch

img_arr = imageio.imread('data/bobby.jpg')

img_arr.shape

# img_array is a numpy array like object with three dimensions: two spatial dimensions, width and height and a third dimension corresponding to the
# red, green and blue channels. 

(720, 1280, 3)

In [2]:
# use the tensors 'permute' method with the old dimensions for each new dimension to get an appropiate layout

# given an input tensor h * w * c we get a proper layout by having channel 2 first and then channels 0 and 1

img = torch.from_numpy(img_arr)
out = img.permute(2,0,1)

# important to note that this operation does not make a copy of the tensor data

In [3]:
# to create a dataset of multiple images to use as an input for neural networks, store the images in a batch along the first dimension
# to obtain a n*c*h*w tensor.

# create a tensor of an appropiate size and fill it with images loaded from a directory

batch_size = 3 
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

# this indicates our batch will consist of three RGB images 256 pixels in height and 256 pixels in length 

# each colour will be represented as an 8 bit integer

In [4]:
import os 
data_dir = 'data/cats/'

filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] =='.png']

for i, filename in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(data_dir,filename))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2,0,1)
    img_t = img_t[:3]   # only keep the first 3 channels, some images have an alpha channel indicating transparency, but this network only wants RGB
    batch[i] = img_t
    

NORMALIZING THE DATA:
neural networks work best with data represented as floating point tensors with a range of [0,1] or [-1,1].

In [5]:
# the easiest way is to divide the values of the pixels by 255 (max. representable number in 8 bit unsigned)
batch = batch.float()
batch /= 255.0

In [11]:
# can normalize the data to mean = 0 and sd = 1
n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:,c])
    std = torch.std(batch[:,c])
    batch[:,c] = (batch[:,c] - mean) / std

3D IMAGES: VOLUMETRIC DATA
CT scans have only a single intensity channel, similar to grayscale images. This means that the channel dimension is left out in native data formats, typically the raw data has three dimensions.

By stacking individual 2d slices into a 3d tensor we can build volumetric data representing the 3d anatomy.

we have a 5d tensor of n * c * d * h * w with d = depth